In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
print("sklearn version: ", sklearn.__version__)
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
!pip install tensorflow_addons
import tensorflow_addons as tfa
from tensorflow import keras
print("TF version: ", tf.__version__)
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# GPU test
print("GPU installed: ",tf.test.is_built_with_gpu_support())

# To prevent "CUDNN_STATUS_ALLOC_FAILED" error with GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")    

from tensorflow.python.keras.utils.data_utils import Sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


sklearn version:  1.0.2
TF version:  2.8.0
GPU installed:  True
4 Physical GPUs, 4 Logical GPUs


2022-06-05 06:59:40.641174: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-05 06:59:43.111525: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2022-06-05 06:59:43.112509: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14791 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-06-05 06:59:43.113239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

In [2]:
X_train=np.uint8(np.load('./X_train_aug.npy'))
y_train=np.uint8(np.load('./y_train_aug.npy'))
X_valid=np.uint8(np.load('./X_valid.npy'))
y_valid=np.uint8(np.load('./y_valid.npy'))

In [3]:
X_valid=tf.reshape(X_valid,[-1,28,28,1])
X_valid_resize=np.uint8(tf.image.resize(X_valid, [32, 32]))
X_train.shape,X_valid_resize.shape,X_train.dtype

((1074263, 32, 32, 1), (164015, 32, 32, 1), dtype('uint8'))

In [4]:
#황성현
class Inveted_Residual_Block(keras.layers.Layer):
    def __init__(self,c,s=1,t=1,n=1,**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.layers.ReLU(max_value=6)
        self.main_layers = [
            
            ]
        self.strides_layers=[
           ]
        self.s=s
        self.t=t
        self.c=c
        self.n=n
        
    def build(self,batch_input_shape):
        print(self.name,": batch_input_shape =",batch_input_shape)
        
        self.main_layers = [
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1, 
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=1,padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c,kernel_size=1, strides=1,padding="SAME", activation=self.activation),
            ]
        self.strides_layers=[
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=batch_input_shape[-1]*self.t, kernel_size=1,strides=1,
                                padding="SAME", activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.DepthwiseConv2D(kernel_size=3, strides=self.s,padding="SAME",activation=self.activation),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(filters=self.c, kernel_size=1,strides=1,padding="SAME",activation=self.activation),
        ]

    def call(self, inputs):
        Z = inputs
        
        for n in range(0,self.n):
            if n == 0:
                for layer in self.strides_layers:
                    Z = layer(Z)
            else:
                skip_Z = Z
                for layer in self.main_layers:
                    Z = layer(Z)
                
                Z += skip_Z
    
        return Z
    
    def get_config(self):
        base=super().get_config()
        return{**base,"s":self.s,"t":self.t,"c":self.c,"n":self.n}

In [9]:
    #황성현
    #multygpu


    n_classes=62
    smv2_l=keras.models.Sequential([
        keras.layers.BatchNormalization(input_shape=[32,32,1]),
        keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding="same",activation="relu"),
        Inveted_Residual_Block(t=1,c=16,n=1,s=1,name="IRB1"),
        Inveted_Residual_Block(t=4,c=16,n=2,s=2,name="IRB2"),
        Inveted_Residual_Block(t=8,c=32,n=2,s=2,name="IRB3"),
        Inveted_Residual_Block(t=8,c=64,n=2,s=2,name="IRB4"),
        Inveted_Residual_Block(t=8,c=128,n=1,s=2,name="IRB5"),
        Inveted_Residual_Block(t=8,c=1024,n=1,s=1,name="IRB6"),
        keras.layers.ReLU(max_value=6),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(n_classes,activation="softmax")
    ])
    smv2_l.summary()
    opt = tf.keras.optimizers.Adam(learning_rate=0.003)
    batch_size=128
    smv2_l.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

IRB1 : batch_input_shape = (None, 16, 16, 32)
IRB2 : batch_input_shape = (None, 16, 16, 16)
IRB3 : batch_input_shape = (None, 8, 8, 16)
IRB4 : batch_input_shape = (None, 4, 4, 32)
IRB5 : batch_input_shape = (None, 2, 2, 64)
IRB6 : batch_input_shape = (None, 1, 1, 128)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  (None, 32, 32, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 IRB1 (Inveted_Residual_Bloc  (None, 16, 16, 16)       2288      
 k)                                                              
                                                                 
 IRB2 (Inveted_Residual_Bloc  (None, 8, 8, 16)   

In [10]:
#황성현 
from datetime import datetime
logs = "logs/" + datetime.now().strftime("SMV2L_lrsch_plat_aug128%Y%m%d-%H%M%S")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
check_best_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_best_lrsch_plat_aug128.h5",save_best_only=True)
check_last_cb=tf.keras.callbacks.ModelCheckpoint(filepath="smv2l_nadam_last_lrsch_plat_aug128.h5",save_best_only=True)
earlystop_cb = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
history=smv2_l.fit(X_train,y_train,validation_data=(X_valid_resize,y_valid),batch_size = batch_size, epochs=300,callbacks=[tboard_callback, check_best_cb, check_last_cb, earlystop_cb,reduce_lr])

2022-06-05 07:23:04.815855: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-05 07:23:04.815904: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-05 07:23:05.528160: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-05 07:23:05.528341: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


Epoch 1/300
 499/8393 [>.............................] - ETA: 2:56 - loss: 2.2112 - accuracy: 0.4303

2022-06-05 07:23:20.070555: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-05 07:23:20.070609: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


 519/8393 [>.............................] - ETA: 3:08 - loss: 2.1676 - accuracy: 0.4393

2022-06-05 07:23:21.525674: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-05 07:23:21.527018: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-05 07:23:21.705110: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 14844 callback api events and 14919 activity events. 
2022-06-05 07:23:21.956645: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-05 07:23:22.268020: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_plat_aug12820220605-072304/plugins/profile/2022_06_05_07_23_21

2022-06-05 07:23:22.501387: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/SMV2L_lrsch_plat_aug12820220605-072304/plugins/profile/2022_06_05_07_23_21/a58a2589147d.trace.json.gz


 525/8393 [>.............................] - ETA: 3:30 - loss: 2.1544 - accuracy: 0.4421

2022-06-05 07:23:22.765167: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/SMV2L_lrsch_plat_aug12820220605-072304/plugins/profile/2022_06_05_07_23_21

2022-06-05 07:23:22.772493: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/SMV2L_lrsch_plat_aug12820220605-072304/plugins/profile/2022_06_05_07_23_21/a58a2589147d.memory_profile.json.gz
2022-06-05 07:23:22.777803: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/SMV2L_lrsch_plat_aug12820220605-072304/plugins/profile/2022_06_05_07_23_21
Dumped tool data for xplane.pb to logs/SMV2L_lrsch_plat_aug12820220605-072304/plugins/profile/2022_06_05_07_23_21/a58a2589147d.xplane.pb
Dumped tool data for overview_page.pb to logs/SMV2L_lrsch_plat_aug12820220605-072304/plugins/profile/2022_06_05_07_23_21/a58a2589147d.overview_page.pb
Dumped tool data for input_pipeline.pb to logs/SMV2L_lrsch_plat_aug128202206

8393/8393 [==============================] - 207s 24ms/step - loss: 0.6715 - accuracy: 0.7787 - val_loss: 0.4779 - val_accuracy: 0.8281 - lr: 0.0030
Epoch 2/300
8393/8393 [==============================] - 201s 24ms/step - loss: 0.4615 - accuracy: 0.8342 - val_loss: 0.4205 - val_accuracy: 0.8488 - lr: 0.0030
Epoch 3/300
8393/8393 [==============================] - 201s 24ms/step - loss: 0.4284 - accuracy: 0.8438 - val_loss: 1.2171 - val_accuracy: 0.6262 - lr: 0.0030
Epoch 4/300
8393/8393 [==============================] - 203s 24ms/step - loss: 0.4113 - accuracy: 0.8491 - val_loss: 1.8235 - val_accuracy: 0.5196 - lr: 0.0030
Epoch 5/300
8393/8393 [==============================] - 200s 24ms/step - loss: 0.4034 - accuracy: 0.8512 - val_loss: 0.4381 - val_accuracy: 0.8363 - lr: 0.0030
Epoch 6/300
8393/8393 [==============================] - 201s 24ms/step - loss: 0.3628 - accuracy: 0.8635 - val_loss: 0.3807 - val_accuracy: 0.8562 - lr: 0.0015
Epoch 7/300
8393/8393 [=======================

In [12]:
smv2_l.save('./smv2l_nadam_lrsch_plat_aug128.h5')